In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from joblib import dump

In [25]:
wolf_rayet = pd.read_csv('archivos/datos finales/catalogo_wr.csv')
muestra_aleatoria = pd.read_csv('archivos/datos finales/muestra_representativa.csv')
muestra_aleatoria.shape

(2000, 219)

In [26]:
print("Wolf rayet:", wolf_rayet.shape)
print(list(wolf_rayet.columns))
print("Muestra aleatoria:", muestra_aleatoria.shape)
print(list(muestra_aleatoria.columns))

Wolf rayet: (333, 36)
['Unnamed: 0', 'coord_index', 'ID', 'WR#', 'RA_deg', 'Dec_deg', 'G', 'query_ra', 'query_dec', 'source_id', 'designation', 'ra', 'dec', 'GAIA_G', 'GAIA_BP', 'GAIA_RP', 'parallax', 'parallax_error', 'distance_deg', 'J', 'H', 'K', 'Distance (kpc)', 'Galactic Longitude (deg)', 'Galactic Latitude (deg)', 'pmra', 'pmdec', 'pmra_error', 'pmdec_error', 'pm', 'GAIA_G-GAIA_BP', 'GAIA_G-GAIA_RP', 'GAIA_BP-GAIA_RP', 'J-H', 'J-K', 'H-K']
Muestra aleatoria: (2000, 219)
['Unnamed: 0', 'source_id', 'solution_id', 'designation', 'source_id.1', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pm', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrome

In [38]:
# Asignare clases a las muestras
wolf_rayet['class'] = 1
muestra_aleatoria['class'] = 0

In [39]:
combined_df = pd.concat([wolf_rayet, muestra_aleatoria], ignore_index=True)
print(combined_df.shape)
print(list(combined_df.columns))

(2333, 232)
['Unnamed: 0', 'coord_index', 'ID', 'WR#', 'RA_deg', 'Dec_deg', 'G', 'query_ra', 'query_dec', 'source_id', 'designation', 'ra', 'dec', 'GAIA_G', 'GAIA_BP', 'GAIA_RP', 'parallax', 'parallax_error', 'distance_deg', 'J', 'H', 'K', 'Distance (kpc)', 'Galactic Longitude (deg)', 'Galactic Latitude (deg)', 'pmra', 'pmdec', 'pmra_error', 'pmdec_error', 'pm', 'GAIA_G-GAIA_BP', 'GAIA_G-GAIA_RP', 'GAIA_BP-GAIA_RP', 'J-H', 'J-K', 'H-K', 'class', 'solution_id', 'source_id.1', 'random_index', 'ref_epoch', 'ra_error', 'dec_error', 'parallax_over_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_pri

In [40]:
combined_df.to_csv('archivos/datos finales/muestra_combinada.csv')

# Oversampling
### Utilizaré modelos de SMOTE-ENN y SMOTE-TOMEK porque leí que limpian parámetros de la frontera en las clases mayoritarias, lo cual creo que puede ser útil debido a que en muchos gráficos color-color las fronteras no están realmente tan claras. 

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
df = pd.read_csv('archivos/datos finales/muestra_combinada.csv')
print(list(df.columns))

['Unnamed: 0.1', 'Unnamed: 0', 'coord_index', 'ID', 'WR#', 'RA_deg', 'Dec_deg', 'G', 'query_ra', 'query_dec', 'source_id', 'designation', 'ra', 'dec', 'GAIA_G', 'GAIA_BP', 'GAIA_RP', 'parallax', 'parallax_error', 'distance_deg', 'J', 'H', 'K', 'Distance (kpc)', 'Galactic Longitude (deg)', 'Galactic Latitude (deg)', 'pmra', 'pmdec', 'pmra_error', 'pmdec_error', 'pm', 'GAIA_G-GAIA_BP', 'GAIA_G-GAIA_RP', 'GAIA_BP-GAIA_RP', 'J-H', 'J-K', 'H-K', 'class', 'solution_id', 'source_id.1', 'random_index', 'ref_epoch', 'ra_error', 'dec_error', 'parallax_over_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric

In [49]:
X = df[['parallax', 'GAIA_G-GAIA_BP', 'GAIA_G-GAIA_RP', 'GAIA_BP-GAIA_RP', 'J-H', 'J-K', 'H-K']] 
y = df['class']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## SMOTE-ENN

In [88]:
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

In [112]:
smote_enn = SMOTEENN(random_state=42)
X_resampled_enn, y_resampled_enn = smote_enn.fit_resample(X_train, y_train)

print(X_resampled_enn.shape, y_resampled_enn.shape)
print(f"Antes de SMOTE_enn: {y_train.value_counts()}")
print(f"Después de SMOTE_enn: {y_resampled_enn.value_counts()}")

(3092, 7) (3092,)
Antes de SMOTE_enn: class
0    1600
1     266
Name: count, dtype: int64
Después de SMOTE_enn: class
1    1569
0    1523
Name: count, dtype: int64


In [122]:
n_estimators = 100
model_enn = XGBClassifier(eval_metric='mlogloss', learning_rate='0.1', objective='binary:logistic', n_estimators=n_estimators, random_state=42)
model_enn.fit(X_resampled_enn, y_resampled_enn)

y_pred_enn = model_enn.predict(X_test)

print("Resultados usando SMOTE-ENN:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_enn)}")
print(classification_report(y_test, y_pred_enn))

with open(f'graficos/XGBOOST/SMOTE_ENN/reporte_enn_{n_estimators}_estimators.txt', 'w') as f:
    f.write("Resultados usando SMOTE-ENN:\n")
    f.write(f"Accuracy: {accuracy_score(y_test, y_pred_enn)}\n")
    f.write("\nReporte de clasificacion:\n")
    f.write(classification_report(y_test, y_pred_enn))

dump(model_enn, f'modelos/xg_enn_{n_estimators}.joblib')

Resultados usando SMOTE-ENN:
Accuracy: 0.9635974304068522
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       400
           1       0.83      0.94      0.88        67

    accuracy                           0.96       467
   macro avg       0.91      0.95      0.93       467
weighted avg       0.97      0.96      0.96       467



['modelos/xg_enn_100.joblib']

In [91]:
cm = confusion_matrix(y_test, y_pred_enn)
print("Matriz de Confusión:")
print(cm)

# Graficar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - SMOTE-ENN')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.savefig(f'graficos/XGBOOST/SMOTE_ENN/matriz_confusion_enn_{n_estimators}_estimators.png')
plt.close()

Matriz de Confusión:
[[387  13]
 [  4  63]]


In [103]:
y_pred_proba = model_enn.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'Curva ROC (área = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1],'r--', label='Clasificación aleatoria')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - SMOTE-ENN')
plt.legend(loc="lower right")
plt.savefig(f'graficos/XGBOOST/SMOTE_ENN/roc_curve_enn_{n_estimators}_estimators.png')
plt.close()

In [106]:
importances = model_enn.feature_importances_
feature_importance_df = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importancia', ascending=False)
print("Características más importantes para la clasificación:")
print(feature_importance_df.head(10))

plt.figure(figsize=(10,8))
plt.barh(feature_importance_df['Característica'], feature_importance_df['Importancia'])
plt.gca().invert_yaxis() 
plt.xlabel('Importancia')
plt.title('Importancia de las Características - SMOTE-ENN')
plt.tight_layout()
plt.savefig(f'graficos/XGBOOST/SMOTE_ENN/feature_importances_enn_{n_estimators}_estimators.png')
plt.close()

Características más importantes para la clasificación:
    Característica  Importancia
6              H-K     0.592830
4              J-H     0.184867
1   GAIA_G-GAIA_BP     0.106621
2   GAIA_G-GAIA_RP     0.036377
0         parallax     0.034914
5              J-K     0.031575
3  GAIA_BP-GAIA_RP     0.012817


### SMOTE-ENN Random forest

In [111]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [123]:
n_estimators = 100
model_rf_enn = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
model_rf_enn.fit(X_resampled_enn, y_resampled_enn)

y_pred_rf_enn = model_rf_enn.predict(X_test)

# Evaluar el modelo
print("Resultados usando Random Forest con SMOTE-ENN:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf_enn)}")
print(classification_report(y_test, y_pred_rf_enn))
dump(model_rf_enn, f'modelos/rf_enn_{n_estimators}.joblib')

# Guardar el reporte en un archivo de texto
with open(f'graficos/RandomForest/SMOTE_ENN/reporte_rf_enn_{n_estimators}_estimators.txt', 'w') as f:
    f.write("Resultados usando Random Forest con SMOTE-ENN:\n")
    f.write(f"Accuracy: {accuracy_score(y_test, y_pred_rf_enn)}\n")
    f.write("\nReporte de clasificacion:\n")
    f.write(classification_report(y_test, y_pred_rf_enn))

# Matriz de confusión
cm_rf_enn = confusion_matrix(y_test, y_pred_rf_enn)
print("Matriz de Confusión:")
print(cm_rf_enn)

plt.figure(figsize=(8,6))
sns.heatmap(cm_rf_enn, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Random Forest con SMOTE-ENN')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.savefig(f'graficos/RandomForest/SMOTE_ENN/matriz_confusion_rf_enn_{n_estimators}_estimators.png')
plt.close()


# Graficar la curva ROC

y_pred_proba_rf = model_rf_enn.predict_proba(X_test)[:,1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

plt.figure(figsize=(8,6))
plt.plot(fpr_rf, tpr_rf, label=f'Curva ROC (área = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Random Forest con SMOTE-ENN')
plt.legend(loc="lower right")
plt.savefig(f'graficos/RandomForest/SMOTE_ENN/roc_curve_rf_enn_{n_estimators}_estimators.png')
plt.close()

#Importancias
importances_rf = model_rf_enn.feature_importances_
feature_importance_rf_df = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importances_rf
})
feature_importance_rf_df = feature_importance_rf_df.sort_values(by='Importancia', ascending=False)

# Mostrar las primeras características más importantes
print("Características más importantes para la clasificación con Random Forest:")
print(feature_importance_rf_df.head(10))

plt.figure(figsize=(10,8))
sns.barplot(x='Importancia', y='Característica', data=feature_importance_rf_df.head(20))
plt.title('Importancia de las Características - Random Forest con SMOTE-ENN')
plt.tight_layout()
plt.savefig(f'graficos/RandomForest/SMOTE_ENN/feature_importances_rf_enn_{n_estimators}_estimators.png')
plt.close()

Resultados usando Random Forest con SMOTE-ENN:
Accuracy: 0.9593147751605996
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       400
           1       0.81      0.94      0.87        67

    accuracy                           0.96       467
   macro avg       0.90      0.95      0.92       467
weighted avg       0.96      0.96      0.96       467

Matriz de Confusión:
[[385  15]
 [  4  63]]
Características más importantes para la clasificación con Random Forest:
    Característica  Importancia
6              H-K     0.490703
0         parallax     0.126234
5              J-K     0.126228
2   GAIA_G-GAIA_RP     0.072203
1   GAIA_G-GAIA_BP     0.064939
3  GAIA_BP-GAIA_RP     0.062546
4              J-H     0.057147


# SMOTE-TOMEK

In [94]:
smote_tomek = SMOTETomek(random_state=42)
X_resampled_tomek, y_resampled_tomek = smote_tomek.fit_resample(X_train, y_train)

print(X_resampled_enn.shape, y_resampled_enn.shape)
print(f"Antes de SMOTE_enn: {y_train.value_counts()}")
print(f"Después de SMOTE_enn: {y_resampled_enn.value_counts()}")

(3092, 7) (3092,)
Antes de SMOTE_enn: class
0    1600
1     266
Name: count, dtype: int64
Después de SMOTE_enn: class
1    1569
0    1523
Name: count, dtype: int64


In [127]:
n_estimators=100
model_tomek = XGBClassifier(eval_metric='mlogloss', learning_rate='0.1', objective='binary:logistic', n_estimators=n_estimators, random_state=42)
model_tomek.fit(X_resampled_tomek, y_resampled_tomek)

y_pred_tomek = model_tomek.predict(X_test)

print("Resultados usando SMOTE-Tomek:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tomek)}")
print(classification_report(y_test, y_pred_tomek))
dump(model_tomek, f'modelos/xg_tomek_{n_estimators}.joblib')


with open(f'graficos/XGBOOST/SMOTE_TOMEK/reporte_tomek_{n_estimators}_estimators.txt', 'w') as f:
    f.write("Resultados usando SMOTE-Tomek:\n")
    f.write(f"Accuracy: {accuracy_score(y_test, y_pred_tomek)}\n")
    f.write("\nReporte de clasificacion:\n")
    f.write(classification_report(y_test, y_pred_tomek))

Resultados usando SMOTE-Tomek:
Accuracy: 0.9678800856531049
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       400
           1       0.84      0.96      0.90        67

    accuracy                           0.97       467
   macro avg       0.92      0.96      0.94       467
weighted avg       0.97      0.97      0.97       467



In [96]:
cm = confusion_matrix(y_test, y_pred_tomek)
print("Matriz de Confusión:")
print(cm)

# Graficar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - SMOTE-TOMEK')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.savefig(f'graficos/XGBOOST/SMOTE_TOMEK/matriz_confusion_tomek_{n_estimators}_estimators.png')
plt.close()

Matriz de Confusión:
[[388  12]
 [  3  64]]


In [97]:
y_pred_proba = model_tomek.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'Curva ROC (área = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - SMOTE-TOMEK')
plt.legend(loc="lower right")
plt.savefig(f'graficos/XGBOOST/SMOTE_TOMEK/roc_curve_tomek_{n_estimators}_estimators.png')
plt.close()

In [108]:
importances = model_tomek.feature_importances_
feature_importance_df = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importancia', ascending=False)
print("Características más importantes para la clasificación:")
print(feature_importance_df.head(10))

plt.figure(figsize=(10,8))
plt.barh(feature_importance_df['Característica'], feature_importance_df['Importancia'])
plt.gca().invert_yaxis()  
plt.xlabel('Importancia')
plt.title('Importancia de las Características - SMOTE-TOMEK')
plt.tight_layout()
plt.savefig(f'graficos/XGBOOST/SMOTE_TOMEK/feature_importances_TOMEK_{n_estimators}_estimators.png')
plt.close()

Características más importantes para la clasificación:
    Característica  Importancia
6              H-K     0.646259
4              J-H     0.118932
1   GAIA_G-GAIA_BP     0.068976
0         parallax     0.051803
3  GAIA_BP-GAIA_RP     0.046551
2   GAIA_G-GAIA_RP     0.035682
5              J-K     0.031796


### RANDOM FOREST TOMEK

In [128]:
n_estimators = 100
model_rf_tomek = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
model_rf_tomek.fit(X_resampled_tomek, y_resampled_tomek)

y_pred_rf_tomek = model_rf_tomek.predict(X_test)

# Evaluar el modelo
print("Resultados usando Random Forest con SMOTE-Tomek:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf_tomek)}")
print(classification_report(y_test, y_pred_rf_tomek))
dump(model_rf_tomek, f'modelos/rf_tomek_{n_estimators}.joblib')

# Guardar el reporte en un archivo de texto
with open(f'graficos/RandomForest/SMOTE_TOMEK/reporte_rf_tomek_{n_estimators}_estimators.txt', 'w') as f:
    f.write("Resultados usando Random Forest con SMOTE-Tomek:\n")
    f.write(f"Accuracy: {accuracy_score(y_test, y_pred_rf_tomek)}\n")
    f.write("\nReporte de clasificacion:\n")
    f.write(classification_report(y_test, y_pred_rf_tomek))

# Matriz de confusión
cm_rf_tomek = confusion_matrix(y_test, y_pred_rf_tomek)
print("Matriz de Confusión:")
print(cm_rf_tomek)

plt.figure(figsize=(8,6))
sns.heatmap(cm_rf_tomek, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Random Forest con SMOTE-Tomek')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.savefig(f'graficos/RandomForest/SMOTE_TOMEK/matriz_confusion_rf_tomek_{n_estimators}_estimators.png')
plt.close()


# Graficar la curva ROC

y_pred_proba_rf = model_rf_tomek.predict_proba(X_test)[:,1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

plt.figure(figsize=(8,6))
plt.plot(fpr_rf, tpr_rf, label=f'Curva ROC (área = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Random Forest con SMOTE-Tomek')
plt.legend(loc="lower right")
plt.savefig(f'graficos/RandomForest/SMOTE_TOMEK/roc_curve_rf_tomek_{n_estimators}_estimators.png')
plt.close()

#Importancias
importances_rf = model_rf_tomek.feature_importances_
feature_importance_rf_df = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importances_rf
})
feature_importance_rf_df = feature_importance_rf_df.sort_values(by='Importancia', ascending=False)

# Mostrar las primeras características más importantes
print("Características más importantes para la clasificación con Random Forest:")
print(feature_importance_rf_df.head(10))

plt.figure(figsize=(10,8))
sns.barplot(x='Importancia', y='Característica', data=feature_importance_rf_df.head(20))
plt.title('Importancia de las Características - Random Forest con SMOTE-Tomek')
plt.tight_layout()
plt.savefig(f'graficos/RandomForest/SMOTE_TOMEK/feature_importances_rf_tomek_{n_estimators}_estimators.png')
plt.close()


Resultados usando Random Forest con SMOTE-Tomek:
Accuracy: 0.961456102783726
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       400
           1       0.83      0.93      0.87        67

    accuracy                           0.96       467
   macro avg       0.91      0.95      0.93       467
weighted avg       0.96      0.96      0.96       467

Matriz de Confusión:
[[387  13]
 [  5  62]]
Características más importantes para la clasificación con Random Forest:
    Característica  Importancia
6              H-K     0.477366
0         parallax     0.134941
5              J-K     0.117670
2   GAIA_G-GAIA_RP     0.070182
4              J-H     0.070143
1   GAIA_G-GAIA_BP     0.066951
3  GAIA_BP-GAIA_RP     0.062747


# SMOTE normal

In [98]:
smote = SMOTE(random_state=42)
X_resampled_smt, y_resampled_smt = smote.fit_resample(X_train, y_train)

print(X_resampled_smt.shape, y_resampled_smt.shape)
print(f"Antes de SMOTE: {y_train.value_counts()}")
print(f"Después de SMOTE: {y_resampled_smt.value_counts()}")

(3200, 7) (3200,)
Antes de SMOTE: class
0    1600
1     266
Name: count, dtype: int64
Después de SMOTE: class
1    1600
0    1600
Name: count, dtype: int64


In [129]:
n_estimators = 100
model_smt = XGBClassifier(eval_metric='mlogloss', learning_rate='0.1', objective='binary:logistic', n_estimators=n_estimators, random_state=42)
model_smt.fit(X_resampled_smt, y_resampled_smt)

y_pred_smt = model_smt.predict(X_test)

print("Resultados usando SMOTE:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_smt)}")
print(classification_report(y_test, y_pred_smt))
dump(model_smt, f'modelos/xg_smt_{n_estimators}.joblib')

with open(f'graficos/XGBOOST/SMOTE/reporte_{n_estimators}_estimators.txt', 'w') as f:
    f.write("Resultados usando SMOTE:\n")
    f.write(f"Accuracy: {accuracy_score(y_test, y_pred_smt)}\n")
    f.write("\nReporte de clasificacion:\n")
    f.write(classification_report(y_test, y_pred_smt))

Resultados usando SMOTE:
Accuracy: 0.9678800856531049
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       400
           1       0.84      0.96      0.90        67

    accuracy                           0.97       467
   macro avg       0.92      0.96      0.94       467
weighted avg       0.97      0.97      0.97       467



In [100]:
cm = confusion_matrix(y_test, y_pred_smt)
print("Matriz de Confusión:")
print(cm)

# Graficar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - SMOTE')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.savefig(f'graficos/XGBOOST/SMOTE/matriz_confusion_smt_{n_estimators}_estimators.png')
plt.close()

Matriz de Confusión:
[[388  12]
 [  3  64]]


In [101]:
y_pred_proba = model_smt.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'Curva ROC (área = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - SMOTE-TOMEK')
plt.legend(loc="lower right")
plt.savefig(f'graficos/XGBOOST/SMOTE/roc_curve_smt_{n_estimators}_estimators.png')
plt.close()

In [ ]:
importances = model_tomek.feature_importances_
feature_importance_df = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importancia', ascending=False)
print("Características más importantes para la clasificación:")
print(feature_importance_df.head(10))

plt.figure(figsize=(10,8))
plt.barh(feature_importance_df['Característica'], feature_importance_df['Importancia'])
plt.gca().invert_yaxis()  
plt.xlabel('Importancia')
plt.title('Importancia de las Características - SMOTE-ENN')
plt.tight_layout()
plt.savefig(f'graficos/XGBOOST/SMOTE_TOMEK/feature_importances_enn_{n_estimators}_estimators.png')
plt.close()

In [110]:
importances = model_smt.feature_importances_
feature_importance_df = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importancia', ascending=False)
print("Características más importantes para la clasificación:")
print(feature_importance_df.head(10))

plt.figure(figsize=(10,8))
plt.barh(feature_importance_df['Característica'], feature_importance_df['Importancia'])
plt.gca().invert_yaxis()  
plt.xlabel('Importancia')
plt.title('Importancia de las Características - SMOTE')
plt.tight_layout()
plt.savefig(f'graficos/XGBOOST/SMOTE/feature_importances_smt_{n_estimators}_estimators.png')
plt.close()

Características más importantes para la clasificación:
    Característica  Importancia
6              H-K     0.643806
4              J-H     0.131588
1   GAIA_G-GAIA_BP     0.070550
0         parallax     0.047281
3  GAIA_BP-GAIA_RP     0.045650
2   GAIA_G-GAIA_RP     0.032490
5              J-K     0.028635


### RANDOM FOREST SMOTE

In [130]:
n_estimators = 100
model_rf_smt = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
model_rf_smt.fit(X_resampled_smt, y_resampled_smt)

y_pred_rf_smt = model_rf_smt.predict(X_test)

# Evaluar el modelo
print("Resultados usando Random Forest con SMOTE:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf_smt)}")
print(classification_report(y_test, y_pred_rf_smt))
dump(model_rf_smt, f'modelos/rf_smt_{n_estimators}.joblib')
# Guardar el reporte en un archivo de texto
with open(f'graficos/RandomForest/SMOTE/reporte_rf_smt_{n_estimators}_estimators.txt', 'w') as f:
    f.write("Resultados usando Random Forest con SMOTE:\n")
    f.write(f"Accuracy: {accuracy_score(y_test, y_pred_rf_smt)}\n")
    f.write("\nReporte de clasificacion:\n")
    f.write(classification_report(y_test, y_pred_rf_smt))

# Matriz de confusión
cm_rf_smt = confusion_matrix(y_test, y_pred_rf_smt)
print("Matriz de Confusión:")
print(cm_rf_smt)

plt.figure(figsize=(8,6))
sns.heatmap(cm_rf_smt, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Random Forest con SMOTE')
plt.ylabel('Etiqueta Verdadera')
plt.xlabel('Etiqueta Predicha')
plt.savefig(f'graficos/RandomForest/SMOTE/matriz_confusion_rf_smt_{n_estimators}_estimators.png')
plt.close()


# Graficar la curva ROC

y_pred_proba_rf = model_rf_smt.predict_proba(X_test)[:,1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

plt.figure(figsize=(8,6))
plt.plot(fpr_rf, tpr_rf, label=f'Curva ROC (área = {roc_auc_rf:.2f})')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Random Forest con SMOTE')
plt.legend(loc="lower right")
plt.savefig(f'graficos/RandomForest/SMOTE/roc_curve_rf_smt_{n_estimators}_estimators.png')
plt.close()

# Importancias
importances_rf = model_rf_smt.feature_importances_
feature_importance_rf_df = pd.DataFrame({
    'Característica': X_train.columns,
    'Importancia': importances_rf
})
feature_importance_rf_df = feature_importance_rf_df.sort_values(by='Importancia', ascending=False)

# Mostrar las primeras características más importantes
print("Características más importantes para la clasificación con Random Forest:")
print(feature_importance_rf_df.head(10))

plt.figure(figsize=(10,8))
sns.barplot(x='Importancia', y='Característica', data=feature_importance_rf_df.head(20))
plt.title('Importancia de las Características - Random Forest con SMOTE')
plt.tight_layout()
plt.savefig(f'graficos/RandomForest/SMOTE/feature_importances_rf_smt_{n_estimators}_estimators.png')
plt.close()

Resultados usando Random Forest con SMOTE:
Accuracy: 0.9657387580299786
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       400
           1       0.84      0.94      0.89        67

    accuracy                           0.97       467
   macro avg       0.91      0.96      0.93       467
weighted avg       0.97      0.97      0.97       467

Matriz de Confusión:
[[388  12]
 [  4  63]]
Características más importantes para la clasificación con Random Forest:
    Característica  Importancia
6              H-K     0.473226
0         parallax     0.138524
5              J-K     0.120700
4              J-H     0.069232
1   GAIA_G-GAIA_BP     0.068039
2   GAIA_G-GAIA_RP     0.067417
3  GAIA_BP-GAIA_RP     0.062862
